# This Jupyter Notebook is written to convert Raw Data and Scores files from NIH Toolbox IPAD exports into NDA data structures using linking information from a 'Crosswalk' and extra NDA-required subject identifier data (GUID, etc) from a csv.

Some Notes: 
using a specialty Python 3 virtual environment (named PycharmToolbox) as kernel for this notebook.
Installed by running the following commands in my terminal and then switching the kernel with the dropdown menu above:
> source /home/petra/.virtualenvs/PycharmToolbox/bin/activate
> pip install ipykernel
> ipython kernel install --user --name=PycharmToolbox
> jupyter-notebook

requirements file generated from within the activated virtual environment by:
> pip freeze > requirements.txt 


In [ ]:
import os, datetime
import pandas as pd
import numpy as np
import subprocess

snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')


Specify the input and output data and paths for NIH toolbox. 
To run the cells of this notebook, you will need four files.

Two are in the .csv format of the IPAD Toolbox applcation export.
E.g. a raw Data file containing scores for item level responses, and a Scores file, containing the summary statistics for the collection of item level data. We don't need the registration file, but it might be handy for filtering out batteries that were inappropriately administered.  These two files are linked by PIN and Inst variables, and must be cleaned a priori to remove subjects that are in one but not the other file.  I.e. the list of unique PINs (ex. HCP0211999_V1) in one file should be exactly the same as the list of unique PINs in the other. For HCP data, we concatenate the exports of all subjects' Score data in to a single file, and the exports of all subjects Raw data into a second file.  Because all other sources of HCP data use 'subject' and 'visit' rather than a PIN which is a concatenation of both, we create these variables (subject and visit) from PIN prior to running this program as well.  

The third necessary file is a csv containing the fields that NDA requires in all of their structures 
e.g. subjectkey (GUID or pseudo-GUID), src_subject_id (e.g. HCP0211999), interview_age (in months), and gender (misnomer for sex assigned at birth).  In HCP data, we link the two sources of information via 'subject' and 'visit.'  

Lastly, read in the crosswalk file - which will map your vars to NDA after transpose is complete.  I have placed the crosswalk from HCP data.  Any instruments in this crosswalk that are the same as yours (look at 'Inst' column) will work for you provided you haven't renamed your columns.  You will have to add any instruments not present, after obtaining variable maps and templates from the NDA for your particular set of NIH Toolbox Data.  

Note that subject and visit are variables we created locally to merge with the data coming from a different local source (REDCap).  They are not variables that are output from the NIH Toolbox app on the Ipads, but are necessary for the merge with the NDA required fields stored elsewhere.


In [ ]:
#########################
#POINT TO YOUR DATA - a scores file and a raw file.  These will be the output from the IPAD, concatenated.  
# E.g. take all the scores and concatenate vertically in to a single file, and take all the raw item data and 
# concatenate into a single file.  Files are linked by PIN and Instrument.  
# In general there will be multiple row in the Raw Data file for every Instrument/PIN row in the Scores file, 
# All exceptions to this generality are accounted for below
##########################

#HCP has two different Lifespan studies going to NDA - HCP - Aging, and HCP Developement.  
#We have a few internal aliases for these studies that need to be labelled here; 
#hcp_studystr gets passed into the functions that name the output structures (e.g. a string for the filename). 
#ndar_studystr gets passed into the argument that opens the csv that contains ALL of the HCP 
#subjects (they are related, so they have to be considered together) and subsets based on study


##########################
#hcp_studystr='yourstudy'
#ndar_studystr='placeholderforastringyouprobablywontneed'
#scoresD='pathtoyourscores/yourscores.csv'
#rawE='pathtoyourrawdata/yourraw.csv'
##########################

#hcp_studystr='HCPD'
#ndar_studystr='HCD'
#scoresD='/home/petra/UbWinSharedSpace1/boxtemp/HCD_Toolbox_Scored_Combined_09_29_2020.csv'
#rawD='/home/petra/UbWinSharedSpace1/boxtemp/HCD_Toolbox_Raw_Combined_09_29_2020.csv'

##########################
hcp_studystr='HCPA'
ndar_studystr='HCA'
scoresD='/home/petra/UbWinSharedSpace1/boxtemp/HCA_Toolbox_Scored_Combined_09_29_2020.csv'
rawD='/home/petra/UbWinSharedSpace1/boxtemp/HCA_Toolbox_Raw_Combined_09_29_2020.csv'



In [ ]:
#path where you want to save your formatted structures
preformatout="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/tlbxIpad_format/"
pathout="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/prepped_structures/"

In [ ]:
#read into dataframe and take a peak (we alphabetized our columns during concat process)
scordata=pd.read_csv(scoresD,header=0,low_memory=False)
#scordata.head()
scordata.subject=scordata.PIN.str[0:10]
len(scordata.PIN.unique())

In [ ]:
rawdata=pd.read_csv(rawD,header=0,low_memory=False,error_bad_lines=True)
#rawdata.head()
rawdata.shape
rawdata.subject=rawdata.PIN.str[0:10]
len(rawdata.PIN.unique())

In [ ]:
#just taking a look here  #unnamed columns result of index not removed from pd.to_csv in other location
rawdata.columns 

In [ ]:
rawdata.PIN.unique()[0:10]
#rawdata.groupby(['Inst','ItemID']).count()


In [ ]:
samplePINS=list(rawdata.PIN.unique()[0:10])
replacePINS=['HCP0000001_V1', 'HCP0000002_V1', 'HCP0000003_V1', 'HCP0000004_V1',
       'HCP0000005_V1', 'HCP0000006_V1', 'HCP0000007_V1', 'HCP0000008_V1',
       'HCP0000009_V1', 'HCP0000010_V1']
pinSUBs=dict(zip(samplePINS,replacePINS))
pinSUBs
sampleRAW=rawdata.loc[rawdata.PIN.isin(samplePINS)]
sampleSCORES=scordata.loc[scordata.PIN.isin(samplePINS)]
sampleRAW=sampleRAW.replace(pinSUBs).drop(columns=['subject','visit'])#rc_subject_id','gender','subjectkey','interview_age','interview_date'])
sampleSCORES=sampleSCORES.replace(pinSUBs).drop(columns=['subject','visit'])#,'src_subject_id','gender','subjectkey','interview_age','interview_date'])
sampleRAW.DateCreated='dummydate'
sampleRAW.InstStarted='dummydate'
sampleRAW.InstEnded='dummydate'
sampleSCORES.DateFinished='dummydate'
sampleRAW.to_csv("SampleRaw.csv",index=False)
sampleSCORES.to_csv("SampleScores.csv",index=False)
#pinSUBs
#sampleSCORES.PIN


In [ ]:
#note that the data still have visits OTHER than v1, which is the only subset we'll be sending
rawdata.groupby('visit').count()


In [ ]:
#subset rawdata and scordata to v1
rawdata=rawdata.loc[rawdata.PIN.str.upper().str.contains('V1')].copy()
scordata=scordata.loc[scordata.PIN.str.upper().str.contains('V1')].copy()
print(len(rawdata.PIN.unique()))
print(len(scordata.PIN.unique()))


In [ ]:
#BEWARE: NIH TOOLBOX IPAD FORMAT FORCES YOU TO LINK DATA BY PIN, WHICH IS PRONE TO CONTAINING TYPOS (no dataset id)
#EVEN AFTER YOU'VE CLEANED THE DATA YOU SHOULD CHECK FOR PROBLEMS... AGAIN...since they can get reintroduced whenever curated data are 'live.'
print("PINS that might cause problems: " + rawdata.loc[rawdata.PIN.str.contains('v1'),'PIN'].unique())
print("PINS that might cause problems: " + scordata.loc[scordata.PIN.str.contains('v1'),'PIN'].unique())
print("Duplicate Alert: "+ rawdata.loc[rawdata.PIN.str.upper().str.contains('HCD2155746_V1'),'PIN'].unique())  #Would be a DUPLICATE if you're counting by unique PINS!!!
rawdata.loc[rawdata.PIN.str.upper().str.contains('HCD2600539_V1'),'PIN'].unique() #not a duplicate, whew
rawdata.loc[rawdata.PIN.str.upper().str.contains('HCD2537457_V1'),'PIN'].unique() #not a duplicate, whew

#remove any duplicates
rawdata=rawdata.loc[~(rawdata.PIN.str.contains('HCD2155746_v1'))]
scordata=scordata.loc[~(scordata.PIN.str.contains('HCD2155746_v1'))]
print(len(rawdata.PIN.unique()))
print(len(scordata.PIN.unique()))


In [ ]:
scordata.groupby('visit').count()

In [ ]:
print("HCA7581081_V1" in rawdata.PIN.unique())
print("HCA7581081_V1" in scordata.PIN.unique())
print("HCA6375275_V1" in rawdata.PIN.unique())
print("HCA6375275_V1" in scordata.PIN.unique())
print("HCA7581081_V1" in rawdata.PIN.unique())
print("HCA7581081_V1" in scordata.PIN.unique())
print("HCA8361577_V1" in rawdata.PIN.unique())
print("HCA8361577_V1" in scordata.PIN.unique())
print("HCA7247574_V1" in rawdata.PIN.unique())
print("HCA7247574_V1" in scordata.PIN.unique())

In [ ]:
#HCP data had some alternatives to '1 or 2' for assessment name that needed to be sent back to RAs for clarification
#after all was said and done - set to missing here after capturing flags because
#fneproc has limitations on character length, and need be consistent
rawdata['Assessment Name']=rawdata['Assessment Name'].str.replace('Assessment ','')
scordata['Assessment Name']=scordata['Assessment Name'].str.replace('Assessment ','')

raw_assessment_strings=rawdata.loc[~(rawdata['Assessment Name'].isin(['1','2','3']))][['PIN','Assessment Name']].drop_duplicates(keep='first')
score_assessment_strings=scordata.loc[~(scordata['Assessment Name'].isin(['1','2','3']))][['PIN','Assessment Name']].drop_duplicates(keep='first')
#score_assessment_strings
#raw_assessment_strings
assessment_strings_forFU=pd.merge(raw_assessment_strings,score_assessment_strings,on=['PIN','Assessment Name'],how='outer',indicator=True)
assessment_strings_forFU.loc[~(assessment_strings_forFU._merge =='both')]
assessment_strings_forFU.drop(columns=['_merge']).to_csv("TLBX Records with Unexpected Assessment Strings.csv",index=False)


#set strings to missing
rawdata.loc[~(rawdata['Assessment Name'].isin(['1','2','3'])),'Assessment Name']=''
scordata.loc[~(scordata['Assessment Name'].isin(['1','2','3'])),'Assessment Name']=''
print(len(rawdata.PIN.unique()))
print(len(scordata.PIN.unique()))


In [ ]:
#prep the fields that NDA requires in all of their structures - we did this in another program, 
#and call it our ROSETTA STONE since output is required elsewhere and has subject aliases in double winner cases.
#Here, just subsetting ROSETTA STONE to particular study (ndar_studystr='HCA' or 'HCD'), 
# renaming a few vars, and changing the date format

#ALSO DROPPING two per Emily A. email 5/28/20 
finaltwo=['HCA8465488','HCA7884605']
#should not have 8_17 batteries: 
another=['HCA9914193']#protocol deviation has <18yo batteries
odorerr=['HCA6623874']#has empty odor battery in curated...not part of hca protocol - send back to trello
griperr=['HCA7130957']#has Grip error wierdness - nih_tlbx_nondomsc = 1.21.2720  row shift? 

subjectlist='/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/UnrelatedHCAHCD_w_STG_Image_and_pseudo_GUID01_15_2021.csv'
subjects=pd.read_csv(subjectlist)
#doublewinners - dont exclude 
#Emily exclusions
subjects=subjects.loc[~(subjects.subjectped.isin(finaltwo))]
#other wierdness
subjects=subjects.loc[~(subjects.subjectped.isin(another))]
subjects=subjects.loc[~(subjects.subjectped.isin(odorerr))]
subjects=subjects.loc[~(subjects.subjectped.isin(griperr))]

subjects.shape

In [ ]:
subjects=subjects[['subjectped','nda_gender', 'nda_guid', 'nda_interview_age', 'nda_interview_date']].copy()
ndar=subjects.loc[subjects.subjectped.str.contains(ndar_studystr)].rename(
    columns={'nda_guid':'subjectkey','subjectped':'src_subject_id','nda_interview_age':'interview_age',
             'nda_interview_date':'interview_date','nda_gender':'gender'}).copy()
ndar['interview_date'] = pd.to_datetime(ndar['interview_date']).dt.strftime('%m/%d/%Y')
ndarlist=['subjectkey','src_subject_id','interview_age','interview_date','gender']


In [ ]:
ndar.columns
ndar.head()

In [ ]:
#this is the list of variables in the scored and raw data files that you might need...
#creating list in case your scored data is merged with other files for other reasons (ours was)
#scorlist=['Age-Corrected Standard Score', 'Age-Corrected Standard Scores Dominant',
# 'Age-Corrected Standard Scores Non-Dominant', 'AgeCorrCrystal', 'AgeCorrDCCS', 'AgeCorrEarly',
# 'AgeCorrEngRead', 'AgeCorrEngVocab', 'AgeCorrFlanker', 'AgeCorrFluid', 'AgeCorrListSort',
# 'AgeCorrPSM', 'AgeCorrPatternComp', 'AgeCorrTotal', 'Assessment Name', 'Computed Score',
# 'ComputedDCCS', 'ComputedEngRead', 'ComputedEngVocab', 'ComputedFlanker', 'ComputedPSM',
# 'ComputedPatternComp', 'DCCSaccuracy', 'DCCSreactiontime',  'Dominant Score', 'FlankerAccuracy',
# 'FlankerReactionTime', 'FullTCrystal', 'FullTDCCS', 'FullTEarly', 'FullTEngRead', 'FullTEngVocab',
# 'FullTFlanker', 'FullTFluid', 'FullTListSort', 'FullTPSM', 'FullTPatternComp', 'FullTTotal',
# 'Fully-Corrected T-score', 'Fully-Corrected T-scores Dominant', 'Fully-Corrected T-scores Non-Dominant',
# 'FullyCorrectedTscore', 'Group', 'Inst', 'InstrumentBreakoff', 'InstrumentRCReason', 'InstrumentRCReasonOther',
# 'InstrumentStatus2', 'ItmCnt', 'Language', 'Male', 'National Percentile (age adjusted)',
# 'National Percentile (age adjusted) Dominant', 'National Percentile (age adjusted) Non-Dominant',
# 'Non-Dominant Score', 'PIN', 'Raw Score Left Ear', 'Raw Score Right Ear', 'RawDCCS',
# 'RawFlanker', 'RawListSort', 'RawPSM', 'RawPatternComp', 'RawScore', 'SE', 'Static Visual Acuity Snellen',
# 'Static Visual Acuity logMAR', 'TScore', 'Theta', 'ThetaEngRead', 'ThetaEngVocab', 'ThetaPSM', 'Threshold Left Ear',
# 'Threshold Right Ear', 'UncorrCrystal', 'UncorrDCCS', 'UncorrEarly', 'UncorrEngRead', 'UncorrEngVocab',
# 'UncorrFlanker', 'UncorrFluid', 'UncorrListSort', 'UncorrPSM', 'UncorrPatternComp', 'UncorrTotal',
# 'Uncorrected Standard Score', 'Uncorrected Standard Scores Dominant', 'Uncorrected Standard Scores Non-Dominant',
# 'UncorrectedStandardScore']
scorlist=['PIN', 'DeviceID', 'Assessment Name', 'Inst',
       'RawScore', 'Theta', 'TScore', 'SE', 'ItmCnt', 'DateFinished',
       'Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Language',
       'Computed Score', 'Uncorrected Standard Score',
       'Age-Corrected Standard Score', 'National Percentile (age adjusted)',
       'Fully-Corrected T-score', 'Uncorrected Standard Scores Dominant',
       'Age-Corrected Standard Scores Dominant',
       'National Percentile (age adjusted) Dominant',
       'Fully-Corrected T-scores Dominant',
       'Uncorrected Standard Scores Non-Dominant',
       'Age-Corrected Standard Scores Non-Dominant',
       'National Percentile (age adjusted) Non-Dominant',
       'Fully-Corrected T-scores Non-Dominant', 'Dominant Score',
       'Non-Dominant Score', 'Raw Score Right Ear', 'Threshold Right Ear',
       'Raw Score Left Ear', 'Threshold Left Ear',
       'Static Visual Acuity logMAR', 'Static Visual Acuity Snellen',
       'InstrumentBreakoff', 'InstrumentStatus2', 'InstrumentRCReason',
       'InstrumentRCReasonOther', 'App Version', 'iPad Version',
       'Firmware Version','Age-Corrected Standard Scores Quinine Whole',
       'Age-Corrected Standard Scores Salt Whole','Fully-Corrected T-scores Quinine Whole',
       'Fully-Corrected T-scores Salt Whole','Uncorrected Standard Scores Quinine Whole',
       'Uncorrected Standard Scores Salt Whole','Whole Mouth Quinine','Whole Mouth Salt']



#rawlist=['App Version', 'Assessment Name', 'DataType','DateCreated', 'DeviceID',  'Firmware Version',  
# 'Inst', 'InstEnded','InstEndedDatetime', 'InstOrdr', 'InstSctn', 'InstStarted','InstStartedDatetime',
# 'ItemID', 'ItmOrdr', 'Locale','PIN', 'Position', 'Response', 'ResponseTime', 'SE', 'Score', 'TScore',
# 'Theta','iPad Version']

rawlist=['App Version', 'Assessment Name', 'DataType','DateCreated', 'DeviceID',  'Firmware Version',  
 'Inst', 'InstEnded','InstOrdr', 'InstSctn', 'InstStarted',
 'ItemID', 'ItmOrdr', 'Locale','PIN', 'Position', 'Response', 'ResponseTime', 'SE', 'Score', 'TScore',
 'Theta','iPad Version']

In [ ]:
#scordata[scorlist]
scordata.columns
#rawdata.columns

In [ ]:
#merge the score and raw data with the required fields for the NDA
#Note that subject and visit are HCP specific variables that we use to subset the records being sent to the NDA
#Depending on how you organized your data, you may need to create dummy vars if you dont have them...
#scordata['subject']=scordata.PIN #or some other variable in scordata that can be used to merge with ndarfields data
#scordata['visit']='V1' #we keep this around because eventually we'll be releaseing V2,V3, and FU data
#rawdata['subject']=rawdata.PIN
#rawdata['visit']='V1'

scordata=pd.merge(scordata[scorlist+['subject','visit']],ndar,how='inner',left_on='subject', right_on='src_subject_id')
rawdata=pd.merge(rawdata[rawlist+['subject','visit']],ndar,how='inner',left_on='subject', right_on='src_subject_id')
print(len(rawdata.PIN.unique()))
print(len(scordata.PIN.unique()))
#scordata=pd.merge(scordata,ndar,how='inner',left_on='subject', right_on='src_subject_id')
#rawdata=pd.merge(rawdata,ndar,how='inner',left_on='subject', right_on='src_subject_id')



In [ ]:
#check for one-to-one match of PINs
for i in scordata.PIN.unique():
    if i not in rawdata.PIN.unique():
        print("not in Raw " + i)
        
for i in rawdata.PIN.unique():
    if i not in scordata.PIN.unique():
        print("not in scores " + i)

In [ ]:

scordata.groupby('visit').count()

#any PINs still have Xs or As and Bs?  if so, drop - rep 2nd parent info...will handle in next release
scordata.loc[scordata.PIN.str.contains('X')]

scordata=scordata.loc[~(scordata.PIN.str.len()>13)]
rawdata=rawdata.loc[~(rawdata.PIN.str.len()>13)]


In [ ]:
#THIS IS THE NUMBER THAT WILL BE ASSOCIATED WITH UPLOAD
print(len(rawdata.PIN.unique()))
print(len(scordata.PIN.unique()))
scordata.loc[scordata.PIN.str.contains("HCA6007044")]

In [ ]:
#Export Ipad formatted version of data to be submitted to NDA (not including erroneous odor and pegboard instruments for HCA)
if hcp_studystr=='HCPA':
    print(rawdata.shape)
    print(scordata.shape)
    rawdata=rawdata.loc[~(rawdata.Inst.isin(['NIH Toolbox Odor Identification Test Age 10+ v2.0','NIH Toolbox 9-Hole Pegboard Dexterity Test Age 3+ v2.0']))]
    scordata=scordata.loc[~(scordata.Inst.isin(['NIH Toolbox Odor Identification Test Age 10+ v2.0','NIH Toolbox 9-Hole Pegboard Dexterity Test Age 3+ v2.0']))]
    print(rawdata.shape)
    print(scordata.shape)
                          
rawdata.to_csv(preformatout+ndar_studystr+'_'+'RAW_TLBX_'+snapshotdate+'.csv',index=False)
scordata.to_csv(preformatout+ndar_studystr+'_'+'SCORES_TLBX_'+snapshotdate+'.csv',index=False)


Do a little QC and data exploration wrt instruments included

In [ ]:
#NOW that you have ALL of your data, take a look at how the intruments are organized within them.  
#for example, the bulk of instruments have representation in the raw files AND the scored files, and can be 
#handled by the 'normal' code block below.  We will need to code for exceptions, though. 
#All code blocks below will skip Practice instruments and Instructions

print('*****Instruments in Raw data but not Scores:')
for i in rawdata.Inst.unique():
    if i not in scordata.Inst.unique():
        print(i)
print('******Instruments in Scored data but not Raw:')
for i in scordata.Inst.unique():
    if i not in rawdata.Inst.unique():
        print(i)

#Occasionally one of the instruments, (NIH Toolbox List Sorting Working Memory Test Ages 3-6 v2.1 in our case) will showing up for protocol
#deviation reasons (someone opened and closed a battery, for example).  The validated column of the crosswalk has been set to NO 
# for this instrument, but you may have the item level info to extend and validate htis in your data.  
#nan rows in Raw data can correspond to cases where Registration Data was uploaded as 'raw' data and concatenated behind the scenes somewhere.  
#you will need to QC (we just allow program to drop them here - send flag back to your data curation team for next release)

In [ ]:
#check that lengths are the same...indicating one to one PIN match between scores and raw
print(len(rawdata.PIN.unique()))
print(len(scordata.PIN.unique()))
#check that shape is same before and after removing duplicates (should not be any)
rawdata.shape
scordata.shape
print(rawdata.shape)
print(scordata.shape)
testraw=rawdata.drop_duplicates(subset={'PIN','Inst','ItemID','Position'},keep='first')
testscore=scordata.drop_duplicates(subset={'PIN','Inst'})
print(testraw.shape)
print(testscore.shape)


In [ ]:
rawdata.columns
#scordata.columns

In [ ]:
#define the function that will turn a prepared (e.g already transformed, renamed, revalued, etc. and otherwise
# ready to go) dataframe into a csv structure 
def data2struct(patho,dout,crosssub,study):
    """
    Convert dout, a prepared pandas dataframe, into a csv structure that NDA can import
    
    parameters: 
    patho - full path to place you want to store structures (there will be many)
    dout - name of data frame that contains all the variables to be exported
    crosssub - a dataframe which is the subset of the crosswalk for the instrument to be exported as structure
    study - a string to put in the name of the csv file along with the structure name and the short name of the instrument
    
    note that snapshotdate is globally defined external to this function near import statments...     
    
    """
    #get the name and number of the structure from the crosswalk subset
    strucroot=crosssub['nda_structure'].str.strip().str[:-2][0]
    strucnum=crosssub['nda_structure'].str.strip().str[-2:][0]
    #prepare the name of the output file and path
    instshort=crosssub['inst_short'].str.strip()[0]
    inst=crosssub['Inst'].str.strip()[0].replace(' ','_').replace('+','plus').replace('-','_') 
    filePath=os.path.join(pathout,study+'_'+instshort+'_'+strucroot+strucnum+'_'+snapshotdate+'.csv')
    if os.path.exists(filePath):
        os.remove(filePath)
    else:
        pass
        #print("Can not delete the file as it doesn't exists")
    with open(filePath,'a') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        dout.to_csv(f,index=False)


In [ ]:
#This function sends a transformed dataframe of the right 'shape' (i.e. after items have been pivoted into row with scores)
# through the crosswalk for renaming, revaluing and structure destination mapping
#function takes a dataframe (in which NIH Toolbox Items are still the names) and formats the column names 
#such that all the special characters are removed because the export has characters that python and the NDA dont like
#This function will alert you to any instruments that were successfully transformed but tha might
#warrent a closer look.

def sendthroughcrosswalk(pathout,instreshapedfull,inst_i,crosswalk,studystr,verbose,debug):
    """
    Send instreshapedfull, a dataframe that has pivoted the item level data into the scored data by instrument
    through the crosswalk to have its variables renamed and reformatted according to the harmonization 
    requests of the NDA
    
    parameters: 
    pathout - full path to place you want to store structures (there will be many) - argument gets passed to data2struct fuctnion
    inst_i - string name of instrument as it appears in the NIH Toolbox output, exactly (case sensitive with version)
    crosswalk - pandas dataframe of crosswalk (read from csv)
    studystr - 'HCPA' or other string specified at the beginning of this notebook - will be passed to data2struct funciton to tag the file
               name with the study source of the data
    verbose - YES or NO, will flag all the variable to element mappings available in the crosswalk 
              that weren't called upon in this transformation because they weren't found in your data
    debug - YES or NO, will print out the last row to be executed from the 'requested_python' column, in case you get an 
            Error, and need to figure out where the loop got stuck
    """
    # replace special charaters in column names
    instreshapedfull.columns = instreshapedfull.columns.str.replace(' ', '_').str.replace('-', '_').str.replace('(','_').str.replace(')', '_')
    crosswalk_subset = crosswalk.loc[crosswalk['Inst'] == inst_i]
    crosswalk_subset.reset_index(inplace=True)
    #
    if crosswalk_subset.reset_index().validated[0]=='NO':
            print("Skipping "+inst_i+ " because crosswalk not yet validated for this instrument ")
    else: 
        # some studies will have some but not all of the variables in hcp_variable (result of skip logic, perhaps)
        # need to make sure they know about this(in case not due to skip logic) but also that code is only execute for
        # vars in existence.  
        #also need to keep track of dummy vars that dont exist in IPAD output, but are necessary for NDA and need to be created 
        #on the fly
        #how many vars are in the instrument according to what is stored in hcp_variable? 
        cwlistbef = list(crosswalk_subset['hcp_variable'])
        before = len(cwlistbef)
        #how many are in the intersection of hcp_variable and the prepared data (e.g. what shows up with the particular instrument in instreshapedfull)
        cwlist = list(set(cwlistbef) & set(
            instreshapedfull.columns))  # drop the handful of vars in larger instruments that got mapped but that we dont have
        after = len(cwlist)
        if before != after:
            print("WARNING!!! " + inst_i + ": Crosswalk expects " + str(before) + " elements, but only found " + str(after)+ " in the prepared data")
            notfound=list(np.setdiff1d(cwlistbef,cwlist))
            if verbose=='YES':
                print("Not Found:"+ str(notfound))
        #get the dummies
        dummys=[]
        for i in cwlistbef:
            if "dummy" in i:
                dummys=dummys+[i]
        #studydata should have all the ndar variables and the list of vars in the intersection of what exists in ipad output and
        #what can be received per the crosswalk.  
        # if you dont force the intersection then code will try to execute on things that dont exist
        #The new dummy variables will be created on the fly.
        studydata = instreshapedfull[ndarlist + cwlist].copy()
        # execute any python one liners (see how they all refer to studydata?) for all rows in the crosswalk corresponding to cwlist this instrument vars except the notfounds.
        itersubset=crosswalk_subset.loc[crosswalk_subset.hcp_variable.isin(cwlist + dummys)]
        for index, row in itersubset.iterrows():#crosswalk_subset.iterrows():
            if pd.isna(row['requested_python']):
                pass
            else:
                if debug=='YES':
                    print(row['requested_python'])
                exec(row['requested_python'])
        uploadlist = list(crosswalk_subset['hcp_variable_upload'])
        uploadlist = list(set(uploadlist) & set(studydata.columns))
        data2struct(patho=pathout, dout=studydata[ndarlist + uploadlist], crosssub=crosswalk_subset, study=studystr)

In [ ]:
#specify your crosswalk- take a peak - use the latest crosswalk from the https://github.com/humanconnectome/NIHToolbox2NDA/
#e.g. Crosswalk_NIH_Toolbox_2_NDA.csv
crosswalkpath="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/"
cfile="Crosswalk_NIH_Toolbox_2_NDA.csv"
crosswalk=pd.read_csv(crosswalkpath+cfile,header=0,low_memory=False, encoding = "ISO-8859-1")
crosswalk.head()


In [ ]:
scordata.columns



In [ ]:
##testing area for problem instruments

#studydata['Raw_Score_Right_Ear'].round().fillna(-9999).astype(int).astype(str).str.replace('-9999','')
#scordata.loc[scordata.Inst=='Social Satisfaction Summary (18+)','TScore']#.round().fillna(-9999).astype(int).astype(str).str.replace('-9999','')
#test=pd.DataFrame(test,columns=['Raw Score Right Ear'])
#test.columns
##test.describe()
#test2=test.loc[~(test['Raw Score Right Ear'].isnull()==True)]
#test2.dtypes
##test['Raw Score Right Ear'].astype(float).round().fillna(-9999).astype(int).astype(str).str.replace('-9999','')
##test2['Raw Score Right Ear'].round()#.fillna(-9999).astype(int).astype(str).str.replace('-9999','')

In [ ]:
for i in scordata.Inst.unique():
    if i not in rawdata.Inst.unique():
        inst_i=i
        print(inst_i)
        
        
 

Do special cases last

In [ ]:
#For cases where instrument is in the scored data but not the raw data, e.g. 
# because this is a summary across instruments, is instructions/practice or because someone 
# opened and closes a battery before generating any item level data 

for i in scordata.Inst.unique():
    if i not in rawdata.Inst.unique():
        inst_i=i
        if "Cognition" in inst_i:
            pass  #special case--see specialty code block below
        elif "Practice" in inst_i:
            print("Note:  Omitting practice instrument, "+inst_i)
        elif "Instructions" in inst_i:
            print("Note:  Omitting Instructions instrument, "+inst_i)
        else:
            try:  #this will fail if there are duplicates or if no-one has the data of interest (e.g. idlist too small), or if only V2 instrument
                #print('Processing '+inst_i+'...')
                instreshapedfull=scordata.loc[scordata.Inst==inst_i][scorlist+ndarlist]
                #instreshapedfull=scordata.loc[scordata.Inst==inst_i].copy()
                instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
                #verbose will tell you which items were not found, debug will print out the python lines before they are executed
                if 'Parent' in inst_i:
                    instreshapedfull['respondent']='caregiver about child subject'
                else:
                    instreshapedfull['respondent']='subject about self'
                sendthroughcrosswalk(pathout,instreshapedfull, inst_i, crosswalk,studystr=hcp_studystr,verbose='YES',debug='NO')
            except:
                print('Couldnt process '+inst_i+'...')

#Lots of dummy variables were needed to fit the IPAD data into the NDA structures without needing a special code block for every
#single structure (see cogcomp01 specialty code below).  The reason for this is that many of these summary scores were coming 
#from different instruments  (NIH Toolbox term) were originally merged into the same structure.  
#basically you're looking for anything but 'dummy' variables in your warnings below to make sure nothing is accidentally omitted 
#in the transformation

In [ ]:
# For some instruments we only want to send scores.  See discussion about Visual Acuity Instruements below, for example. 
# For other instruments we only CAN send scores, 
# one reason is that only score level data is avaialbe for this instrument (see next code cell)
# Other reason is because only score level data is ready to go (item level data not yet mapped at the NDA)
# for example, the Picture Sequence instruments...NIH Data dictionary wasn't sufficient to 
# map the item level data to the NDA, where items are incorrect/correct but NIH Toolbox outputting value range of
# 0::14. Item level detail for this particular instrument will require facilitation between NIH Toolbox and NDA to define
# new variables for these particular items (not done yet). Similar story for Words-In-Noise, where data dictionar at both ends
# (NIH Toolbox and NDA expectation) doesn't match the observed output.  Yay.  
# When it is done, remove the elif statement from the non-special 
# cases cell block below
# Note also you'll get a lot of warnings for these because you're only sending scores when items are also (almost) available
scoresonly=scordata.loc[(scordata.Inst.str.contains('Visual Acuity')==True) | (scordata.Inst.str.contains('Picture Sequence Memory Test')==True)| (scordata.Inst.str.contains('Words-In-Noise')==True)]
for i in scoresonly.Inst.unique():
    inst_i=i
    if "Practice" in inst_i:
        print("Note:  Omitting practice instrument, "+inst_i)
    else:    
        try:  #this will fail if there are duplicates or if no-one has the data of interest (e.g. idlist too small), or if only V2 instrument
            #print('Processing '+inst_i+'...')
            instreshapedfull=scordata.loc[scordata.Inst==inst_i][scorlist+ndarlist]
            #instreshapedfull=scordata.loc[scordata.Inst==inst_i].copy()
            instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
            #verbose will tell you which items were not found, debug will print out the python lines before they are executed
            if 'Parent' in inst_i:
                instreshapedfull['respondent']='caregiver about child subject'
            else:
                instreshapedfull['respondent']='subject about self'
            sendthroughcrosswalk(pathout,instreshapedfull, inst_i, crosswalk,studystr=hcp_studystr,verbose='NO',debug='NO')
        except:
            print('Couldnt process '+inst_i+'...')



In [ ]:
#for non-special instruments in both scores AND raw data types (skip the scoresonly ones from above)
#add a check to make sure that everything found in the data has a row in the crosswalk
#add indicator for whether this is a variable from the scores file or a variable from the raw data file.
for i in scordata.Inst.unique():
    if i in rawdata.Inst.unique():
        inst_i=i
        if "Visual Acuity" in inst_i:
            pass  #special case--see below
        elif "Practice" in inst_i:
            print("Note:  Omitting practice instrument, "+inst_i)
        #new elif statements here needed because of unresolved discrepancey between NIH Data Dictionary and observed output
        elif "Picture Sequence Memory Test" in inst_i:
            print("Note:  Sent Scores only for "+inst_i)
        elif "Words-In-Noise" in inst_i:
            print("Note:  Sent Scores only for "+inst_i)
        else:
            try:  #this will fail if there are duplicates or if no-one has the data of interest (e.g. idlist too small), or if only V2 instrument
                #print('Processing '+inst_i+'...')
                items=rawdata.loc[rawdata.Inst==inst_i][['PIN','subject','Inst','visit','ItemID','Position',
                   'subjectkey','src_subject_id','interview_age','interview_date','gender',
                   'Score','Response','ResponseTime']]# not these..., 'SE', 'Response', 'TScore','Theta']]
                items.ItemID = items.ItemID.str.lower().str.replace('-','_').str.replace('(','_').str.replace(')','_').str.replace(' ','_')
                inst=items.pivot(index='PIN',columns='ItemID',values='Score').reset_index()
                meta=items.drop_duplicates(subset=['PIN','visit'])
                instreshaped = pd.merge(meta, inst, on='PIN', how='inner').drop(columns={'subject', 'visit','Inst'})
                items2=scordata.loc[scordata.Inst==inst_i][scorlist]
                instreshapedfull=pd.merge(instreshaped,items2,on='PIN',how='inner')
                instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
                #verbose will tell you which items were not found, debug will print out the python lines before they are executed
                if 'Parent' in inst_i:
                    instreshapedfull['respondent']='caregiver about child subject'
                else:
                    instreshapedfull['respondent']='subject about self'
                sendthroughcrosswalk(pathout,instreshapedfull, inst_i, crosswalk,studystr=hcp_studystr,verbose='YES',debug='NO')
            except:
                print('ERROR: Couldnt process '+inst_i+'...')

# NOTE: its okay if there are items in the crosswalk that doent exist in your data...lots of reasons, but please compare
# with Ipad formatted version to make sure nothing is wierd (e.g. no problems with crosswalk which will lowercase the item names from the NIH Toolbox Ipad 'item' column
# for example if a crosswalk can transform elements that are found in an instrument for 3-6 yos, but you dont have any 3-6 yos, then youll be alerted
# not okay if there are scores in the crosswalk that doent exsit in the data...need to investigate
# also will be flags for dummy vars because they exist in crosswalk but not data (they were created for the NDA)
# turn verbose off to ignore these warnings.  Errors will still be reported.
# for instruments not validated--check crosswalk. v2.1 of some isntruments dont exist in the NIH Toolbox Data
# Dictionary yet, so its pointless to try to pretend that we know what they map to in the NDA.  

In [ ]:
#special coding required for instruments in the crosswalk that have jupyter in their specialty code columns
#Within the rawdata structure (for HCP), all but the NIH Toolbox Pain Intensity FF Age 18+ v2.0 Instrument are practices
#So only the Pain Intensity instrument needed special coding attention 
#check your data and adjust if needed - note that subject and visit are variables we created locally 
#to merge with the data coming from a different local source (REDCap)
#create the NDA structure for this special case
#this structure doesnt get 'sent through crosswalk' so any code that is in python column wont get executed

inst_i='NIH Toolbox Pain Intensity FF Age 18+ v2.0'
#most of the rows contain duplicated information...only need to know the PIN once, for example, not once for each item response
# so values in the response column need to be pivoted and then merged with the rest of the data, 
paindata=rawdata.loc[rawdata.Inst==inst_i][['PIN','subject','Inst','visit','ItemID','Position',
        'subjectkey','src_subject_id','interview_age','interview_date','gender',
        'Response','ResponseTime', 'SE', 'Score', 'TScore','Theta','Assessment Name']]
paindata.ItemID = paindata.ItemID.str.lower().str.replace('-','_').str.replace('(','_').str.replace(')','_')
inst = paindata.pivot(index='PIN', columns='ItemID', values='Score').reset_index()
meta = paindata.drop_duplicates(subset=['PIN', 'visit'])

#meta['Inst']=inst_i
painreshaped = pd.merge(meta, inst, on='PIN', how='inner').drop(columns={'subject','visit','PIN'})
crosswalk_subset=crosswalk.loc[crosswalk['Inst']==inst_i]
crosswalk_subset.reset_index(inplace=True)
cwlist=list(crosswalk_subset['hcp_variable_upload']) #these should all correspond with the nda_element names in this structure

#several dummy vars for required vars - normally these would have placeholders in the meta (scores) files but 
#since this particular instrument only exists in the raw data, we have to explicitly create place holders
painreshaped['pssr8_12_10']=painreshaped.pssr8_12_10.round().fillna(-9999).astype(int).astype(str).str.replace('-9999','')
painreshaped['nih_tlbx_agegencsc']=999
painreshaped['nih_tlbx_rawscore']=999
painreshaped['nih_tlbx_tscore']=999
painreshaped['nih_tlbx_se']=999
painreshaped['nih_tlbx_theta']=999
painreshaped['respondent']='self'
painreshaped['version_form']=painreshaped['Inst']
painreshaped['fneproc']=painreshaped['Assessment Name'].str.replace('Assessment ','')
painreshaped['comqother']=painreshaped['respondent']

#painreshaped['version_form']=painreshaped.Inst
#+','+painreshaped['Assessment Name']

reshapedslim=painreshaped[ndarlist+cwlist]

#the data2struct function only uses the crosswalk to get the structure name and number for the header of dout
#dout is otherwise ready to go and data2structure just writes it to a file in the specified location
data2struct(patho=pathout,dout=reshapedslim,crosssub=crosswalk_subset,study=hcp_studystr)

In [ ]:
# Another special case is for Cognition Composite scores all v1.1 - going to cogcomp01 structure at the NDA- 
# Cog comp is special for several reason...it doesnt have corresponding entries in the raw data because
# it represents a summary across instruments in the Cognitive domain.  Even so, 4 cog comp 'instruments' are going to 
# a single NDA structure.  Each of these insturments has a version number and an assessmen
# This was mapped before Leo agreed to accept data by NIH Toolbox Instrument name (pivot by Inst)
# keeping this special case coding in for posterity and to shed light on one type of merge he must do on his end
# and the fact that this special situation is not yet being addressed (unless they can take multiple rows per person)
# Note that this structure illustrates the versioning problem when merging several NIH Toolbox Instruments together
# onto the same row
# Instruments are being mapped to the same # buckets
# One of the main issues that will hopefully be resolved by teleconference 3/23 is how the NDA is keeping track of 
# several instruments (with different versions) getting mapped to the same rows in a structure
# when it comes to NIH toolbox data

cogcompdata=scordata.loc[scordata.Inst.str.contains('Cognition')==True][['PIN','Language',
    'Assessment Name','Inst',  'Uncorrected Standard Score', 'Age-Corrected Standard Score',
    'National Percentile (age adjusted)', 'Fully-Corrected T-score']+ndarlist]

#initialize prefix
cogcompdata['varprefix']='test'
cogcompdata.loc[cogcompdata.Inst=='Cognition Crystallized Composite v1.1','varprefix']='nih_crystalcogcomp_'
cogcompdata.loc[cogcompdata.Inst=='Cognition Early Childhood Composite v1.1','varprefix']='nih_eccogcomp_'
cogcompdata.loc[cogcompdata.Inst=='Cognition Fluid Composite v1.1','varprefix']='nih_fluidcogcomp_'
cogcompdata.loc[cogcompdata.Inst=='Cognition Total Composite Score v1.1','varprefix']='nih_totalcogcomp_'

#pivot the vars of interest by varprefix and rename
uncorr=cogcompdata.pivot(index='PIN',columns='varprefix',values='Uncorrected Standard Score')
for col in uncorr.columns.values:
    uncorr=uncorr.rename(columns={col:col+"unadjusted"})
ageadj=cogcompdata.pivot(index='PIN',columns='varprefix',values='Age-Corrected Standard Score')
for col in ageadj.columns.values:
    ageadj=ageadj.rename(columns={col:col+"ageadj"})
npage=cogcompdata.pivot(index='PIN',columns='varprefix',values='National Percentile (age adjusted)')
for col in npage.columns.values:
    npage=npage.rename(columns={col:col+"np_ageadj"})

#put them together
cogcompreshape=pd.concat([uncorr,ageadj,npage],axis=1)

#hijacking what is the same for all four instruments
meta=cogcompdata[['PIN','Language']+ndarlist].drop_duplicates(subset={'PIN'})

#all the data in place
cogcompreshape=pd.merge(meta,cogcompreshape,on='PIN',how='inner')

# Now grabbing version and assessment info for version_form
# initial attempt to capture the version failed ...they got mapped to raw scores 
# and failed validation 
# per email, all of this information will go to the 'version_form' variable.
meta2=cogcompdata[['PIN','Inst','Assessment Name']].drop_duplicates(subset={'PIN','Inst'})
meta2['Inst,Assessment Name']=meta2['Inst']+','+meta2['Assessment Name']
meta3=meta2.pivot(index='PIN',columns='Inst',values='Inst,Assessment Name')
#this will only work until there are more than one versions of the composites in the data
#need to make it more flexible so that there are 4 possible instruments (whatever version they may be)
meta3['version_monster']=meta3['Cognition Crystallized Composite v1.1']+';'+meta3['Cognition Early Childhood Composite v1.1']+';'+meta3['Cognition Fluid Composite v1.1']+';'+meta3['Cognition Total Composite Score v1.1']

meta3['nih_crystalcogcomp']=meta3['Cognition Crystallized Composite v1.1']
meta3['nih_eccogcomp']=meta3['Cognition Early Childhood Composite v1.1']
meta3['nih_fluidcogcomp']=meta3['Cognition Fluid Composite v1.1']
meta3['nih_totalcogcomp']=meta3['Cognition Total Composite Score v1.1']

##for i in meta3.columns.to_list();  
#meta3=meta3['version_form'].reset_index()

cogcompreshape=pd.merge(cogcompreshape,meta3,on='PIN',how='inner')
#cogcompreshape.columns
inst_i='Cognition Composite Scores'  #one instrument here...is merging of four instruments there

#crosswalk_subset=crosswalk.loc[crosswalk.Inst==inst_i]
#cwlist=list(crosswalk_subset['hcp_variable_upload']) #these should all correspond with the nda_element names in this structure
#reshapedslim=cogcompreshape[ndarlist+cwlist]
#cogcompreshape[cwlist]
#the data2struct function only uses the crosswalk to get the structure name and number for the header of dout
#dout is otherwise ready to go and data2structure just writes it to a file in the specified location
#data2struct(patho=pathout,dout=reshapedslim,crosssub=crosswalk_subset,study=hcp_studystr)


sendthroughcrosswalk(pathout,cogcompreshape,inst_i,crosswalk,studystr=hcp_studystr,verbose='No',debug='Yes')

In [ ]:
filePath="./prepped_structures"
for f in os.listdir(filePath):
    with open(filePath+'/'+f) as g:
        first_line = g.readline()
        print(f)
        print(first_line, end ="")
    df=pd.read_csv(filePath+'/'+f,header=1)
    print("NumRows: "+str(df.shape[0]))
    print()
    print()
    

In [ ]:
##test area for testing individual instruments
inst_i='NIH Toolbox Dimensional Change Card Sort Test Ages 3-7 v2.1'
items=rawdata.loc[rawdata.Inst==inst_i][['PIN','subject','Inst','visit','ItemID','Position',
   'subjectkey','src_subject_id','interview_age','interview_date','gender',
   'Score','ResponseTime']]# not these..., 'SE', 'Score', 'TScore','Theta']]
items.ItemID = items.ItemID.str.lower().str.replace('-','_').str.replace('(','_').str.replace(')','_').str.replace(' ','_')
inst=items.pivot(index='PIN',columns='ItemID',values='Score').reset_index()
meta=items.drop_duplicates(subset=['PIN','visit'])
instreshaped = pd.merge(meta, inst, on='PIN', how='inner').drop(columns={'subject', 'visit','Inst'})
items2=scordata.loc[scordata.Inst==inst_i][scorlist]

instreshapedfull=pd.merge(instreshaped,items2,on='PIN',how='inner')
instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
if 'Parent' in inst_i:
    instreshapedfull['respondent']='caregiver about child subject'
else:
    instreshapedfull['respondent']='subject about self'
#for i in instreshapedfull.columns:
#    print(i)
#meta.columns
##items2.columns

In [ ]:
#Visual Acuity instruments, for example, have an unknown and variable number of 
#repeated items at different 'positions' which would require a double transpose into a single instrument 'row' per person.
#NDA mapped all the different positions we saw in our data (see placeholders in crosswalk), but will assuredly 
#not have all of your positions.  For visual acuity, the scores matter more than the individual items, however, 
#so we opted to only send scores for this particular instrument.  Feel free to extend the specialty code below to 
#accommodate item level instrument for visual acuity.  
#Visual Acuity items not yet mapped - come back to it if possible before the release otherwise omit item levels this release
#scores are mapped in the first special case above.


#Last special Case is for Visual Acuity, which needs double pivot because of repeat items at different positions
#This special case not yet mapped by NDA - so don't run, but will look something like this
#special case for instruments with "Visual Acuity" in their titles, which have dup inst/itemid at diff positions
#for i in scordata.Inst.unique():
#    if i in rawdata.Inst.unique():
#        inst_i=i
#        if "Visual Acuity" in inst_i:
#            print('Processing ' + inst_i + '...')
#                items=rawdata.loc[rawdata.Inst.str.contains('Visual Acuity')][['PIN','subject','Inst',
#                   'gender','visit','ItemID','Position','Response','Score']]
#                items.ItemID = items.ItemID.str.lower()
#               items['dup_number']=items.groupby(['PIN','ItemID']).cumcount()+1
#               items['ItemID_Dup']=items.ItemID.str.replace('|', '_') + '_P'+items.dup_number.astype(str)
#               inst=items.pivot(index='PIN',columns='ItemID_Dup',values='Score')
#               meta = items.drop_duplicates(subset=['PIN', 'visit'])[['Inst', 'PIN', 
#                                                              'subject', 'visit']]
#               instreshaped = pd.merge(meta, inst, on='PIN', how='inner')
#               items2 = scordata.loc[scordata.Inst == inst_i]
#               instreshapedfull = pd.merge(instreshaped, items2, on='PIN', how='inner')

In [ ]:
#now lets see what we have.  
whatdata=pd.DataFrame(rawdata.PIN.unique(),columns={'PIN'})
print(whatdata.shape)
##import list from Cindy for Plasma folks
#plasma=pd.read_csv("20200723_list for plasma HCP2.0.csv", header=None)
#plasma.columns=['subject']
#plasma['Plasma']='YES'
whatdata['subject']=whatdata.PIN.str[0:10]
whatdata['V1_TLBX']='YES'
#whatdata=pd.merge(whatdata, plasma, on='subject',how='outer')#.drop(columns='PIN')
whatdata.loc[whatdata.V1_TLBX.isnull()==True,'V1_TLBX']='NO'
##whatdata.loc[whatdata.Plasma.isnull()==True,'Plasma']='NO'
##whatdata.head()
##whatdata.shape
##whatdatdata.loc[whatdata.Plasma.isnull()==True,'Plasma']='NO'
##a.to_csv(preformatout+ndar_studystr+'_'+'Whatdata_TLBX_and_Plasma_NDA_HCA2.0'+snapshotdate+'.csv',index=False)
whatdata.head()

In [ ]:
#for all the structures in the prepped diretories, create an indicator for subject included
pathout="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/prepped_structures/"
pathout2="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/"
structs=pd.DataFrame(os.listdir(pathout),columns=['fname'])
#structs.to_csv(pathout+'/listoffiles.csv')
print(whatdata.shape)
def getsubjects(filestruct=structs.fname[0], path=pathout, what=whatdata):
    din=pd.read_csv(path+filestruct,header=1)
    print(filestruct)
    dout=pd.DataFrame(din.src_subject_id.unique(),columns={'src_subject_id'}) 
    #print(dout.columns)
    try: 
        doutinst=pd.DataFrame(din.version_form.unique(),columns={'version_form'})
        dout[doutinst.version_form[0]]='1' 
    except:
        print('exception: '+ filestruct)
        if 'cogcomp' in filestruct:
            dout['Cognition Composite Instruments']='1'
    #what=whatdata.copy()
    #print(dout.columns)
    what=pd.merge(what,dout,left_on='subject',right_on='src_subject_id',how='outer')#.drop(columns=['src_subject_id'])
    what=what.drop(columns=['src_subject_id'])
    #print(what.columns)
    return what

for i in structs.fname:
    #whatdata=getsubjects(filestruct=structs.fname[0], path=pathout, what=whatdata)
    whatdata=getsubjects(filestruct=i, path=pathout, what=whatdata)
print(whatdata.shape)
structs.shape
whatdata.head()
#whatdata['src_subject_id']=whatdata.PIN
#whatdata.to_csv(preformatout+ndar_studystr+'_'+'Whatdata_TLBXINST_and_Plasma_NDA_HCA2.0'+snapshotdate+'.csv',index=False)
whatdata.to_csv(pathout2+ndar_studystr+'_'+'Whatdata_TLBXINST_'+snapshotdate+'.csv',index=False)


In [ ]:
print(pathout2+ndar_studystr+'_'+'Whatdata_TLBXINST_'+snapshotdate+'.csv')

now validate all of these files by calling the OS from within this notebook (assuming you are using linux) to run the NDA validator on your command line.  Alternatively, you could just navigate to your terminal and
execute the following for loop .  

for var in pathout/*.csv; do vtcmd $var; done

Either option requires that you have downloaded and installed https://github.com/NDAR/nda-tools python package
per instructions.  I installed vtcmd in my home directory, which set a couple defaults in place., such as the location of validation results. To have the output of the validation sent to a more meaningful location than than the default, I opened the /home/petra/.NDATools/settings.cfg file, and  
changed the line under [Files] that says 'validation_results = NDAValidationResults' to a better place (perhaps 'pathout').  Example, mine now says 
validation_results = /home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/NDAValidationResults

so that the prepped structures directory and the NDAValidationResults Directory are right next to one another.


If you had an error in the validation, your likely course of action is to debug the python code in the the crosswalk. Here are some unix commands to help filter through common issues in the Validation results:  

Find all the not integer warnings:
grep notInteger /home/petra/NDAValidationResults/* > Notintegerwarnings

Find all the invalid range warnings 
grep "invalid" NDAValidationResults/* | cut -d ',' -f 1,6

Cat all of the validation results together so you can see them all at once
cat NDAValidationResults/validation*.csv > NDAValidationResults/Allvalidations.csv 

In [ ]:
structs=pd.DataFrame(os.listdir(pathout),columns=['fname'])
structs.fname.str.split('_')
#structs.to_csv(pathout+'/listoffiles.csv')
